# First I download the necessary libraries

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors

! pip install folium==0.5.0
import folium # plotting library

print('\n Libraries Downloaded!')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes

 Libraries Downloaded!


# Next, I set up to make the soup

In [4]:
# First, the parameters 
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
html=requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
locgeo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')
print("Parameters set")

Parameters set


### Extracting the table from the site

In [14]:
table = soup.findAll('table')
trs = table[0].findAll('tr')

# helper method to extracting all information from a list of 'tr' tag inside table
def find_all_element(trs):
    result = []
    for tr in trs:
        td = tr.findAll('td')
        # if tr contain td tags other than 3, continue
        if len(td) !=3:
            continue
        postcode = td[0].text.replace('\n','')
        borough = td[1].text.replace('\n','')
        
        # only processing data other than 'Not assigned'
        if borough == 'Not assigned':
            continue
        neigh = td[2].text.replace('\n','')
        if neigh == 'Not assigned':
            neigh = borough
        geo_coord = locgeo_df[locgeo_df.index==postcode]
        
        ## get coordinate
        l1 = np.float(geo_coord['Latitude'])
        l2 = np.float(geo_coord['Longitude'])
        buffer_dict = {'POST_CODE':postcode,'BOROUGH':borough, 'neighbour':neigh.strip(),'Latitude':l1,'Longitude':l2}
        result.append(buffer_dict)
    return pd.DataFrame(result)  

df = find_all_element(trs)
df

,POST_CODE,BOROUGH,neighbour,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [9]:
df.shape

(103, 5)